In [1]:
import pandas as pd

# 엑셀 파일 불러오기
file_path1 = r'C:\Users\LeeHwanJu\Desktop\3m_1211개.xlsx'
file_path2 = r'C:\Users\LeeHwanJu\Desktop\4m_1381개.xlsx'
file_path3 = r'C:\Users\LeeHwanJu\Desktop\5m_1140개.xlsx'
file_path4 = r'C:\Users\LeeHwanJu\Desktop\6m_1148개.xlsx'
file_path5 = r'C:\Users\LeeHwanJu\Desktop\7m_1351개.xlsx'

sheet_name = 'Collected data' #바꿀필요X
excel_file1 = pd.read_excel(file_path1, sheet_name=sheet_name)   #3m
excel_file2 = pd.read_excel(file_path2, sheet_name=sheet_name)   #4m
excel_file3 = pd.read_excel(file_path3, sheet_name=sheet_name)   #5m
excel_file4 = pd.read_excel(file_path4, sheet_name=sheet_name)   #6m
excel_file5 = pd.read_excel(file_path5, sheet_name=sheet_name)   #7m

# 원하는 열 선택
column_name = 'RSSI'  # 가져올 열의 이름 바꿀필요X
limit = 1000  # 가져올 열의 개수 제한
column_data1 = excel_file1[column_name][:limit].tolist()  # 열 데이터를 리스트로 변환하고, 개수 제한
column_data2 = excel_file2[column_name][:limit].tolist()  # 열 데이터를 리스트로 변환하고, 개수 제한
column_data3 = excel_file3[column_name][:limit].tolist()  # 열 데이터를 리스트로 변환하고, 개수 제한
column_data4 = excel_file4[column_name][:limit].tolist()  # 열 데이터를 리스트로 변환하고, 개수 제한
column_data5 = excel_file5[column_name][:limit].tolist()  # 열 데이터를 리스트로 변환하고, 개수 제한

In [2]:
def apply_median_filter(values, window_size = 3): #중간값 필터링 함수
    len_values = len(values)
    filtered_values = []
    window_size = max(1, min(window_size, len_values))
    if window_size % 2 != 0:
        window_half = window_size // 2
        for i in range(len_values):
            window = values[max(0, 1 - window_half): min(i + window_half + 1, len_values)]
            filtered_values.append(sorted(window)[window_half])
    else:
        window_half = window_size // 2
        for i in range(len_values):
            window = values[max(0, i - window_half + 1): min(i + window_half + 1, len_values)]
            filtered_values.append((sorted(window)[window_half - i] + sorted(window)[window_half]) / 2)
    return filtered_values

In [3]:
filtered_data1=apply_median_filter(column_data1)     # filtered data 는 열 데이터를 중간값 필터링한것
filtered_data2=apply_median_filter(column_data2)
filtered_data3=apply_median_filter(column_data3)
filtered_data4=apply_median_filter(column_data4)
filtered_data5=apply_median_filter(column_data5)



In [4]:
import torch                                     # 필터링된 열 데이터를  tensor로 변환하고 행x열 로 치환
import numpy as np
rssi_list1 = torch.Tensor(filtered_data1)
rssi_list1=rssi_list1.reshape(1000,1)     
rssi_list2 = torch.Tensor(filtered_data2)
rssi_list2=rssi_list2.reshape(1000,1) 
rssi_list3 = torch.Tensor(filtered_data3)
rssi_list3=rssi_list3.reshape(1000,1) 
rssi_list4 = torch.Tensor(filtered_data4)
rssi_list4=rssi_list4.reshape(1000,1) 
rssi_list5 = torch.Tensor(filtered_data5)
rssi_list5=rssi_list5.reshape(1000,1) 

In [5]:
# 여러 개의 Tensor를 수직으로 쌓기
stacked_rssi = torch.vstack((rssi_list1,rssi_list2,rssi_list3,rssi_list4,rssi_list5))

In [6]:
label = np.zeros((1, 5000), dtype=int) #label 만드는 코드

label_list = []
for i in range(3, 8):
    label_list += [i] * 1000 #이 부분이 각 미터당 몇개의 행으로 할건지.


label=torch.Tensor(label_list)
np.shape(label)
label=label.reshape(-1,1)
label


tensor([[3.],
        [3.],
        [3.],
        ...,
        [7.],
        [7.],
        [7.]])

In [7]:
import numpy as np

# 데이터셋과 레이블셋을 같은 인덱스로 섞기
permutation = np.random.permutation(stacked_rssi.shape[0])
shuffled_dataset = stacked_rssi[permutation]
shuffled_labels = label[permutation]

# 섞인 데이터셋과 레이블셋 확인
print("Shuffled Dataset:\n", shuffled_dataset)
print("Shuffled Labels:\n", shuffled_labels)

Shuffled Dataset:
 tensor([[-81.],
        [-74.],
        [-73.],
        ...,
        [-77.],
        [-81.],
        [-74.]])
Shuffled Labels:
 tensor([[3.],
        [7.],
        [5.],
        ...,
        [3.],
        [3.],
        [7.]])


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random

# 데이터를 훈련셋과 검증셋, 테스트셋으로 분할합니다.
X_train = torch.tensor(shuffled_dataset[:2500], dtype=torch.float32)
y_train = torch.tensor(shuffled_labels[:2500], dtype=torch.float32)
X_val = torch.tensor(shuffled_dataset[2500:4000], dtype=torch.float32)
y_val = torch.tensor(shuffled_labels[2500:4000], dtype=torch.float32)
X_test = torch.tensor(shuffled_dataset[4000:], dtype=torch.float32)
y_test = torch.tensor(shuffled_labels[4000:], dtype=torch.float32)

# RNN 모델을 정의합니다.
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.rnn1 = nn.RNN(input_size=1, hidden_size=64, num_layers=1, batch_first=True) # 첫 번째 RNN 레이어
        self.dropout1 = nn.Dropout(0.2) # 첫 번째 드롭아웃 레이어
        self.rnn2 = nn.RNN(input_size=64, hidden_size=32, num_layers=1, batch_first=True) # 두 번째 RNN 레이어
        self.dropout2 = nn.Dropout(0.2) # 두 번째 드롭아웃 레이어
        self.rnn3 = nn.RNN(input_size=32, hidden_size=16, num_layers=1, batch_first=True) # 두 번째 RNN 레이어
        self.dropout3 = nn.Dropout(0.2) 
        self.fc = nn.Linear(16, 1)
        
    def forward(self, x):
        x, _ = self.rnn1(x)
        x = self.dropout1(x)
        x, _ = self.rnn2(x)
        x = self.dropout2(x)
        x, _ = self.rnn3(x)
        x = self.dropout3(x)
        x = self.fc(x[:, -1, :])  # Use only the last output of the RNN
        return x

   
model = RNN()

# 손실 함수와 옵티마이저를 정의합니다.
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.2)

# 모델을 훈련합니다.
for epoch in range(1000):
    optimizer.zero_grad()
    outputs = model(X_train.unsqueeze(2))  # Add an additional dimension for the RNN input
    loss = criterion(outputs.squeeze(), y_train)  # Remove the extra dimension from the output for loss calculation
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{1000}], Loss: {loss.item():.4f}')
        
    # 검증셋으로 모델 성능 평가
    with torch.no_grad():
        val_outputs = model(X_val.unsqueeze(2))
        val_loss = criterion(val_outputs.squeeze(), y_val)
        if (epoch+1) % 100 == 0:
            print(f'Validation Loss: {val_loss.item():.4f}')

# 테스트셋을 이용하여 거리를 예측합니다.
with torch.no_grad():
    y_pred = model(X_test.unsqueeze(2))

# 예측된 거리를 출력합니다
y_pred = torch.tensor(y_pred.reshape(-1, 1), dtype=torch.float32)
y_test = torch.tensor(y_test.reshape(-1, 1), dtype=torch.float32)

print("Predicted Values (y_pred):")
print(y_pred)

print("True Values (y_test):")
print(y_test)

C:\Users\LeeHwanJu\AppData\Local\Temp\ipykernel_10720\2566227397.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(shuffled_dataset[:2500], dtype=torch.float32)
C:\Users\LeeHwanJu\AppData\Local\Temp\ipykernel_10720\2566227397.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(shuffled_labels[:2500], dtype=torch.float32)
C:\Users\LeeHwanJu\AppData\Local\Temp\ipykernel_10720\2566227397.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_val = torch.tensor(shuffled_dataset[2500:4000], dtype

Epoch [100/1000], Loss: 2.1533
Validation Loss: 2.0735
Epoch [200/1000], Loss: 2.0623
Validation Loss: 1.9878
Epoch [300/1000], Loss: 2.0281
Validation Loss: 1.9547
Epoch [400/1000], Loss: 2.0191
Validation Loss: 1.9471
Epoch [500/1000], Loss: 2.0177
Validation Loss: 1.9452
Epoch [600/1000], Loss: 2.0175
Validation Loss: 1.9450
Epoch [700/1000], Loss: 2.0175
Validation Loss: 1.9449
Epoch [800/1000], Loss: 2.0175
Validation Loss: 1.9449
Epoch [900/1000], Loss: 2.0175
Validation Loss: 1.9449
Epoch [1000/1000], Loss: 2.0175
Validation Loss: 1.9449
Predicted Values (y_pred):
tensor([[4.9888],
        [4.9888],
        [4.9888],
        [4.9888],
        [4.9888],
        [4.9888],
        [4.9888],
        [4.9888],
        [4.9888],
        [4.9888],
        [4.9888],
        [4.9888],
        [4.9888],
        [4.9888],
        [4.9888],
        [4.9888],
        [4.9888],
        [4.9888],
        [4.9888],
        [4.9888],
        [4.9888],
        [4.9888],
        [4.9888],
        

C:\Users\LeeHwanJu\AppData\Local\Temp\ipykernel_10720\2566227397.py:67: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_pred = torch.tensor(y_pred.reshape(-1, 1), dtype=torch.float32)
C:\Users\LeeHwanJu\AppData\Local\Temp\ipykernel_10720\2566227397.py:68: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test = torch.tensor(y_test.reshape(-1, 1), dtype=torch.float32)


In [10]:
# 예상되는 거리를 출력합니다
input_values = torch.tensor([[-30.]], dtype=torch.float32)
with torch.no_grad():
    predicted_values = model(input_values.unsqueeze(2))
print(input_values)
print("Predicted Values:")
print(predicted_values)


tensor([[-30.]])
Predicted Values:
tensor([[4.9888]])
